# Đồ án cuối kỳ - Phân tích chủ đề văn bản
(Cập nhật 07/01/2021)

Nhóm: 12

Thành viên nhóm:
- Vũ Đăng Hoàng Long - MSSV: 18120203
- Nguyễn Huỳnh Đại Lợi - MSSV: 18120198

---
## Import thư viện

In [4]:
!pip3 install pandas
!pip3 install tqdm
!pip3 install bs4
!pip3 install -U scikit-learn
!pip3 install regex
!pip3 install numpy
import pandas as pd
import regex as re
import time # Dùng để sleep chương trình
from tqdm.notebook import tqdm # Hiện thanh progress cho đẹp :D

# Thư viện để lấy và parse HTML
import requests
from bs4 import BeautifulSoup

# Các thư viện liên quan tới ngôn ngữ và NLP
!pip3 install pyvi ;
!pip3 install gensim ;
!pip3 install underthesea ;
from pyvi import ViTokenizer # thư viện NLP tiếng Việt
from underthesea import word_tokenize
import gensim
import unicodedata

# Thư viện liên quan đến machine learning của Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer

Requirement already up-to-date: scikit-learn in /home/vlozg/.local/lib/python3.8/site-packages (0.24.0)


---
## 1. Thu thập dữ liệu

Dữ liệu trong đồ án này được thu thập toàn bộ từ trang báo mạng Tuổi Trẻ Online (https://tuoitre.vn/).

Khi mới vào trang chủ, có thể thấy trang báo này phân bổ thứ tự và cấu trúc thông tin không đều, rất khó để có thể thu thập được. Tuy nhiên, may mắn là trang báo này có mục tin mới nhất chứa toàn bộ tin từ các chuyên mục và có tổ chức các mục có cấu trúc rõ ràng (Hình 1).

![Hình 1](resources/tin-moi-nhat.png "Hình 1")

Khi người dùng kéo xuống cuối trang, sẽ có nút "Xem thêm" (Hình 2). Người dùng nhấn vào đây và tin tức cũ hơn sẽ hiện ngay bên dưới để người dùng kéo xuống tiếp.

![Hình 2](resources/xem-them.png "Hình 2")

Thoáng nhìn qua, quy trình lướt tin tức sẽ bao gồm: Vào mục tin mới -> Kéo xuống cuối trang -> Bấm "Xem thêm" -> Kéo xuống cuối trang và lặp lại. Như vậy với cách làm đơn giản nhất là mô phỏng click chuột của người dùng để bấm nút xem thêm càng nhiều càng tốt, sau đó lấy file HTML của trang về và thực hiện parse (Đây chính là cách làm trong phiên bản đầu của quy trình này của nhóm). Tuy nhiên cách này rất thiếu hiệu quả bởi lẽ máy sẽ cần phải mở trình duyệt lên và mô phỏng thao tác của người dùng -> rất chậm do thời gian tải trang và tốn tài nguyên xử lý của máy tính do phải render trang web. Trong phiên bản đầu nhóm đã mất khoảng vài phút để có thể lấy được 480 tin.

Nhận thấy rằng trang tin này có một đặc điểm là có thể lăn chuột vô tận, nhóm tìm hiểu và phát hiện rằng có thể cải thiện hiệu suất của quy trình này lên rất nhiều lần! Cụ thể, khi người dùng bấm vào nút "Xem thêm", trang web sẽ gửi yêu cầu lên server và nhận về một file HTML có cấu trúc đơn giản chứa thông tin các bài báo (Hình 3). Sau đó trình duyệt sẽ thực hiện thay đổi DOM của trang web để thêm các tin này vào bên dưới, tạo hiệu ứng lăn chuột vô tận. Do vậy, thay vì phải đợi trình duyệt render trang web, nhóm có thể request thẳng đường link giống cách trang web request lên server. Việc làm này sẽ rút ngắn rất nhiều thời gian của quá trình thu thập dữ liệu!

![Hình 3](resources/xem-them-network.png "Hình 3")

Khi sử dụng công cụ theo dõi network của trình duyệt, nhóm phát hiện ra rằng đường link request tin mới của trang web khi bấm nút xem thêm lần đầu là "https://tuoitre.vn/timeline/0/trang-1.htm". Rất có thể khi thay thế trang-1 thành trang-2, trang-3,... chúng ta sẽ thu được các kết quả là tin mới của các lần bấm thứ 2, thứ 3,... vào nút "Xem thêm" (điều này đúng, nhóm đã kiểm chứng).

Đoạn code bên dưới sẽ thực hiện request vào đường link trên và thực hiện parse với BeautifulSoup như HTML bình thường để lấy đường link và chuyên mục của các tin tức. Sau đó sẽ thực hiện request trang tin chính từ đường link thu thập được và lấy các thông tin tiêu đề, mô tả và nội dung.

Kết quả sẽ được in bằng hàm tail() giúp có thể nắm được số lượng data lấy được thông qua index.

In [3]:
# Thiết lập đường dần cho phần 1
dir_1 = "src/scraped_data_demo"

In [56]:
# Thử nghiệm url đầu tiên
url = "https://tuoitre.vn/timeline/0/trang-1.htm"

# Lấy danh sách news_items
html_text = requests.get(url).text
html_tree = BeautifulSoup(html_text, 'html.parser')
news_items = html_tree.findAll('li', class_='news-item')

# Lấy ra link, title và category từ news_items
raw_data = pd.DataFrame(columns=["links","title","description","content","class"])
for item in news_items:
    title = item.find('h3', class_='title-news').text.replace('\n','')
    link = "https://tuoitre.vn" + item.find('a').attrs["href"]
    category = item.find('a', class_='category-name').text
    raw_data = raw_data.append({"links":link, "title":title, "class":category}, ignore_index=True)

raw_data.tail()

,links,title,description,content,class
15,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",NaN,NaN,Sức khỏe
16,https://tuoitre.vn/lang-kinh-24g-canh-giac-voi...,Lăng kính 24g: Cảnh giác với tội phạm trộm cắp...,NaN,NaN,Cần biết
17,https://tuoitre.vn/game-show-cuoi-cung-chi-tai...,Gameshow cuối cùng của Chí Tài - Cơ hội đổi đờ...,NaN,NaN,Giải trí
18,https://tuoitre.vn/giua-kho-khan-thoi-covid-19...,"Giữa khó khăn 'thời COVID-19', một doanh nghiệ...",NaN,NaN,Thời sự
19,https://tuoitre.vn/triet-pha-duong-day-mang-th...,"Triệt phá đường dây mang thai hộ, mỗi lần 'đẻ ...",NaN,NaN,Pháp luật


In [57]:
%%time
for _, row in raw_data.iterrows():
    news_page = requests.get(row["links"]).content
    news_tree = BeautifulSoup(news_page, "html.parser")
    # Lấy mô tả
    try:
        row["description"] = news_tree.find("h2", class_="sapo").text
    except:
        row["description"] = ''
    # Lấy nội dung
    try:
        body = news_tree.find("div", id="main-detail-body")
        content = body.findChildren("p", recursive=False)
        row["content"] = ""
        for x in content:
            row["content"] += x.text
    except:
        row["content"] = ''
    time.sleep(0.05)
    
raw_data.tail()

Wall time: 2.83 s


,links,title,description,content,class
15,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",TTO - Sau nhiều ngày số mắc mới khá cao và dồn...,"Theo Bộ Y tế, 1 ca mắc mới ngày 6-1 là ca nhập...",Sức khỏe
16,https://tuoitre.vn/lang-kinh-24g-canh-giac-voi...,Lăng kính 24g: Cảnh giác với tội phạm trộm cắp...,,,Cần biết
17,https://tuoitre.vn/game-show-cuoi-cung-chi-tai...,Gameshow cuối cùng của Chí Tài - Cơ hội đổi đờ...,"TTO - Tối 5-1, tập đầu tiên của chương trình t...",Cơ hội đổi đời là chương trình truyền hình thự...,Giải trí
18,https://tuoitre.vn/giua-kho-khan-thoi-covid-19...,"Giữa khó khăn 'thời COVID-19', một doanh nghiệ...",TTO - Hầu hết doanh nghiệp tại Đồng Nai duy tr...,"Ngày 6-1, Sở Lao động - thương binh và xã hội ...",Thời sự
19,https://tuoitre.vn/triet-pha-duong-day-mang-th...,"Triệt phá đường dây mang thai hộ, mỗi lần 'đẻ ...",TTO - Công an quận Long Biên (Hà Nội) vừa triệ...,"Ngày 6-1, Công an quận Long Biên (Hà Nội) cho ...",Pháp luật


### Vấn đề phát sinh trong quá trình thu thập dữ liệu tự động hàng loạt

#### Thiếu dữ liệu làm đứt quãng quá trình thu thập
Do khối lượng dữ liệu có thể thu thập được rất lớn (ước tính lên đến hơn 500.000 tin), việc giám sát quá trình khai thác liên tục là không thể. Việc xuất hiện lỗi này đã khiến quá trình khai thác bị ngắt quãng mỗi khi xuất hiện. Nhóm đã khắc phục bằng cách đặt các khối lệnh ```try``` ```catch``` ở mỗi đoạn code parse các phần thông tin, tự động trả về rỗng nếu có lỗi.

#### Lỗi TooManyRedirects: Exceeded 30 redirects
Đây là lỗi xuất hiện khi một trang web tự điều hướng quá 30 lần (con số mặc định có thể chỉnh được).

Với một số server, khi được request sẽ tùy vào thông tin header của request (như trình duyệt, cache,...), server sẽ đính kèm lệnh chuyển hướng vào response và trình duyệt sẽ dùng thông tin đó điều hướng chuyển qua trang web nhất định. Chẳng hạn, khi người dùng điện thoại truy cập vào trang facebook.com, trình duyệt sẽ tự động điều hướng chuyển người dùng tới trang m.facebook.com là trang có giao diện dành cho điện thoại. Request HTTP trong python cũng không phải ngoại lệ khi có thiết lập mặc định là sẽ tự động chuyển hướng cho đến khi không còn nhận được lệnh từ server.

Khi tra cứu lỗi này trên google, nhóm nhận được các đề xuất giải quyết sau:
- Thêm tham số ```allow_redirects=True``` vào lệnh ```requests.get```
- Thêm tham số ```headers``` vào lệnh ```requests.get```. Tham số này chứa các thông tin mặc định của header mà trình duyệt gửi tới server.
- Tạo một ```session``` và thay đổi thuộc tính ```max_redirect``` lên hớn 30.

Với cách 1 vô nghĩa do tham số đó là mặc định sẵn, cách 3 thì không đảm bảo thay đổi bao nhiêu là đủ. Còn với cách 2, lệnh request cung cấp thêm thông tin cho server nhằm tránh tình trạng server không xác định được người dùng và rơi vào vòng lặp redirect vô tận. Tuy nhiên nhóm áp dụng và vẫn không khắc phục được. Khi thử lấy link này thì hoàn toàn không vô được.

Một điều thú vị là khi debug, nhóm phát hiện rằng các link sinh ra lỗi này đều có nội dung về giáo dục (Hình 4). Khi thử tìm kiếm trang riêng có đường link https://tuyensinh.tuoitre.vn/ thì được điều hướng trở về trang https://tuoitre.vn/giao-duc/tuyen-sinh.htm. Kết luận của nhóm: rất có thể trang báo này đã có quá trình thay đổi công nghệ ở một thời điểm nhất định trong quá khứ khiến cho các trang này không truy cập được -> không chỉ trang tin giáo dục mà còn có thể nhiều trang tin khác cũng bị tương tự.

![Hình 4](resources/tuyen-sinh-errors.png "Hình 4")

Quyết định của nhóm là sẽ áp dụng cách khắc phục 2 vì nó hợp lý và áp khối ```try``` ```catch``` điền thông tin trống với các link tiếp tục bị lỗi.

#### Lỗi 502 của trang web được request về
Lỗi này đến từ phía server của trang báo và xuất hiện rất ngẫu nhiên không xác định được. Nhóm đã giải quyết bằng cách đặt vòng lặp ```while``` tại các câu lệnh ```get``` và chỉ ngừng khi quá số lần cho phép hoặc nhận phản hồi 200 tức OK. Tuy nhiên cần chú ý cho phép cả phản hồi 301 vì đây là lệnh điều hướng trang có liên quan tới lỗi ở phần trước có đề cập.

### Thiết lập đoạn code thu thập dữ liệu tự động hàng loạt

In [59]:
def single_request_scraping(index = 1, sleep_time = 0.05):
    '''
    Thu thập các trang tin trong mục tin mới nhất của báo Tuổi Trẻ.
    Mỗi lần thu thập 20 tin. Sử dụng số index để thu thập các trang liên tiếp.
    Với index = 1 tức lấy 20 trang tin mới nhất.

    Lưu ý:
        Khi lấy liên tục cần chọn thời điểm
        vì có khả năng báo cập nhật tin tức mới sẽ làm 
        tin ở trang trước bị đẩy xuống trang sau.

    Param:
        index: số chỉ trang cần request.
    '''
    allow_status = [200, 301]
    s = requests.Session()
    s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    url = f"https://tuoitre.vn/timeline/0/trang-{index}.htm"

    # Lấy danh sách news_items
    try:
        response = s.get(url)
        try_left = limit_retry
        while (response.status_code not in allow_status and try_left > 0):
            print(f"Loi {response.status_code} tai trang {index}")
            response = s.get(url)
            try_left -= 1
        html_text = response.text
    except:
        print(f"Loi request tai trang {index}")
        return None
    
    html_tree = BeautifulSoup(html_text, 'html.parser')
    news_items = html_tree.findAll('li', class_='news-item')
    
    # Kiểm tra lỗi nếu không lấy được bất cứ item nào
    if (len(news_items) == 0):
        print(f'Trang {index} khong lay duoc item.')
        print(html_text)
        return None
    elif (len(news_items) != 20):
        print(f"Warning: Trang {index} chi lay duoc {len(news_items)} items.")
    
    # Lấy ra link, title và category từ news_items
    raw_data = pd.DataFrame(columns=["links","title","description","content","class"])
    for item in news_items:
        try:
            title = item.find('h3', class_='title-news').text.replace('\n','')
        except:
            title = ""
        link = "https://tuoitre.vn" + item.find('a').attrs["href"]
        try:
            category = item.find('a', class_='category-name').text
        except:
            category = ""
        raw_data = raw_data.append({"links":link, "title":title, "class":category}, ignore_index=True)
    
    # Tiến hành lấy nội dung từng link
    for _, row in raw_data.iterrows():
        try:
            response = s.get(row["links"])
            try_left = limit_retry
            while (response.status_code not in allow_status and try_left > 0):
                print(f"Loi {response.status_code} tai link {row['links']}")
                response = s.get(row["links"])
                try_left -= 1
            news_page = response.content
        except:
            print(f"Loi request tai link {row['links']}")
            row["description"] = ""
            row["content"] = ""
            continue
            
        news_tree = BeautifulSoup(news_page, "html.parser")
        # Lấy mô tả
        try:
            row["description"] = news_tree.find("h2", class_="sapo").text
        except:
            row["description"] = ''
        # Lấy nội dung
        try:
            body = news_tree.find("div", id="main-detail-body")
            content = body.findChildren("p", recursive=False)
            row["content"] = ""
            for x in content:
                row["content"] += x.text
        except:
            row["content"] = ''
        time.sleep(sleep_time)
    
    return raw_data

In [52]:
#Batch scraping
iter_num = 1  # Số batch bắt đầu
num = 200  # Số trang link trong 1 batch
continue_flag = True # Cờ hiệu kết thúc vòng lặp khi xảy ra lỗi

while (continue_flag):
    '''
    Khởi tạo dataframe rỗng. 
    Sau đó lấy đủ 1 số trang cho 1 batch.
    Rồi export file csv.
    '''
    batch_df = pd.DataFrame(columns=["links","title","description","content","class"])
    for index in range(iter_num*num+1,(iter_num+1)*num+1):
        data = single_request_scraping(index,0)
        if (data is None):
            continue_flag = False
            break
        print(f"Page {index} complete!")
        batch_df = batch_df.append(data)
    batch_df.to_csv(f'scraped_data\crawling_{iter_num}.csv',index=False,encoding="utf-8")    
    iter_num+=1

Page 3401 complete!
Page 3402 complete!
Page 3403 complete!
Page 3404 complete!
Page 3405 complete!
Page 3406 complete!
Page 3407 complete!
Page 3408 complete!
Page 3409 complete!
Page 3410 complete!
Page 3411 complete!
Page 3412 complete!
Page 3413 complete!
Page 3414 complete!
Page 3415 complete!
Page 3416 complete!
Page 3417 complete!
Page 3418 complete!
Page 3419 complete!
Page 3420 complete!
Page 3421 complete!
Page 3422 complete!
Page 3423 complete!
Page 3424 complete!
Page 3425 complete!
Page 3426 complete!
Page 3427 complete!
Page 3428 complete!
Page 3429 complete!
Page 3430 complete!
Page 3431 complete!
Page 3432 complete!
Page 3433 complete!
Page 3434 complete!
Page 3435 complete!
Page 3436 complete!
Page 3437 complete!
Page 3438 complete!
Page 3439 complete!
Page 3440 complete!
Page 3441 complete!
Page 3442 complete!
Page 3443 complete!
Page 3444 complete!
Page 3445 complete!
Page 3446 complete!
Page 3447 complete!
Page 3448 complete!
Page 3449 complete!
Page 3450 complete!


FileNotFoundError: [Errno 2] No such file or directory: 'scraped_data\\crawling_17.csv'

---
## 2. Khám phá dữ liệu

Trong phần này nhóm sẽ khám phá các đặc tính của dữ liệu đã thu thập được ở bên trên. Do khối lượng dữ liệu lớn, nhóm chỉ đọc 12 batchs dữ liệu đầu tiên để khám phá và đưa ra hướng cho tiền xử lý. Một số thông tin thống kê cần chạy trên toàn bộ dữ liệu sẽ được thực hiện riêng và cho kết quả ra một file riêng biệt.

### Đọc 12 file đầu

In [2]:
# Thiết lập đường dẫn thư mục để dùng trong phần 2
dir_2 = "src/raw_scraped_data/"

In [3]:
# Import thử 12 file đầu tiên
df = []
for i in tqdm(range(12)):
    df.append(pd.read_csv(dir_2 + f'crawling_{i}.csv'))
    
data_df=pd.concat(df)
data_df.reset_index(drop=True, inplace=True)
data_df.head()

,links,title,description,content,class
0,https://tuoitre.vn/tong-thong-trump-xac-nhan-k...,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,https://tuoitre.vn/dat-nuoc-dat-niem-tin-vao-n...,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,https://tuoitre.vn/luat-su-my-phan-bien-ong-tr...,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,https://tuoitre.vn/thanh-pho-phu-quoc-se-phat-...,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,https://tuoitre.vn/ong-trump-nguoi-ung-ho-toi-...,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới


### Kích thước và kiểu dữ liệu của các cột

Theo ước tính trong bước thu thập, file đầu tiên ("crawling_0.csv") sẽ có 8000 dòng còn các file còn lại sẽ có 4000 dòng.<br>Tổng cộng tất cả là 8000 ✕ 1 + 4000 ✕ 11 = 52000. Về kiểu dữ liệu thì các cột sẽ có kiểu string.

In [4]:
data_df.shape

(52000, 5)

In [5]:
# Các cột hiện có kiểu dữ liệu gì?
data_df.dtypes

links          object
title          object
description    object
content        object
class          object
dtype: object

### Kiểm tra dữ liệu trùng

In [6]:
data_df[data_df.duplicated(keep=False)].sort_values(by=['links'])

,links,title,description,content,class
48880,/dang-sai-lech-vu-dong-tam-facebooker-chuong-m...,"Đăng sai lệch vụ Đồng Tâm, Facebooker ‘Chương ...","TTO - Ngày 20-1, Cơ quan cảnh sát điều tra Côn...","Facebooker ""Chương May Mắn"" tên thật là Chung ...",Pháp luật
48879,/dang-sai-lech-vu-dong-tam-facebooker-chuong-m...,"Đăng sai lệch vụ Đồng Tâm, Facebooker ‘Chương ...","TTO - Ngày 20-1, Cơ quan cảnh sát điều tra Côn...","Facebooker ""Chương May Mắn"" tên thật là Chung ...",Pháp luật
307,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",TTO - Sau nhiều ngày số mắc mới khá cao và dồn...,"Theo Bộ Y tế, 1 ca mắc mới ngày 6-1 là ca nhập...",Sức khỏe
254,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",TTO - Sau nhiều ngày số mắc mới khá cao và dồn...,"Theo Bộ Y tế, 1 ca mắc mới ngày 6-1 là ca nhập...",Sức khỏe
179,https://tuoitre.vn/affordable-luxury-dong-san-...,'Affordable luxury' - dòng sản phẩm đột phá về...,Apec Mandala Wyndham Mũi Né đánh dấu bước đầu ...,Apec Group và thương hiệu bất động sản mang tí...,Cần biết
...,...,...,...,...,...
229,https://tuoitre.vn/vuong-quoc-anh-ghi-nhan-hon...,Vương quốc Anh ghi nhận hơn 1.000 người chết m...,TTO - Vương quốc Anh ngày 6-1 ghi nhận thêm 1....,"Theo hãng tin Reuters, trong ngày 6-1, Vương q...",Sức khỏe
304,https://tuoitre.vn/wall-street-journal-bac-kin...,Wall Street Journal: Bắc Kinh ép Jack Ma chia ...,TTO - Chính quyền Trung Quốc đang cố gắng buộc...,Báo Wall Street Journal nhận định Jack Ma có r...,Kinh doanh
251,https://tuoitre.vn/wall-street-journal-bac-kin...,Wall Street Journal: Bắc Kinh ép Jack Ma chia ...,TTO - Chính quyền Trung Quốc đang cố gắng buộc...,Báo Wall Street Journal nhận định Jack Ma có r...,Kinh doanh
286,https://tuoitre.vn/xu-phat-3-nguoi-75-trieu-do...,Xử phạt 3 người 75 triệu đồng vì khai thác kho...,TTO - Chủ đất và hai người tham gia lấy phần c...,"Ngày 6-1, Công an tỉnh Trà Vinh tống đạt các q...",Pháp luật


Các dữ liệu trùng có index cách nhau không quá lớn. Như vậy rất có thể xuất hiện dữ liệu trùng là do trang web cập nhật bài báo mới, làm dữ liệu trong lúc thu thập bị đẩy xuống gây trùng lắp.

### Kiểm tra dữ liệu thiếu

In [7]:
data_df.isna().sum()

links            0
title          244
description    292
content        908
class            0
dtype: int64

In [8]:
data_df[data_df.isna().any(axis=1)].head()

,links,title,description,content,class
29,https://tuoitre.vn/lang-kinh-24g-xe-du-ben-coc...,"Lăng kính 24h: 'Xe dù, bến cóc' - vấn nạn nhiề...",NaN,NaN,Cần biết
123,https://tuoitre.vn/doc-bao-cung-ban-8-1-viet-n...,Đọc báo cùng bạn 8-1: Việt Nam sẽ có vắc xin d...,TTO - Chương trình hôm nay có nhiều thông tin ...,NaN,Bạn đọc làm báo
129,https://tuoitre.vn/video-daisuke-matsui-the-hi...,Video Daisuke Matsui thể hiện đẳng cấp ngay lầ...,NaN,NaN,Thể thao
137,https://tuoitre.vn/duoc-hlv-huynh-duc-kem-chat...,"Được HLV Huỳnh Đức 'kèm chặt', Đức Chinh liền ...","TTO - Chiều 7-1, tiền đạo Hà Đức Chinh đã gây ...",NaN,Thể thao
166,https://tuoitre.vn/lang-kinh-24g-doi-no-thue-h...,Lăng kính 24g: Đòi nợ thuê 'hết đất' lộng hành,NaN,NaN,Cần biết


Dữ liệu có vẻ bị thiếu ngẫu nhiên ở mỗi thuộc tính. Tuy nhiên thuộc tính *Content* bị thiếu nhiều nhất trong khi đây cũng là thuộc tính quan trọng nhất. Do vậy, hướng tiền xử lý các dữ liệu bị thiếu là loại bỏ tất cả các dòng có dữ liệu bị thiếu.

---
## 3. Tiền xử lý bước đầu

Trong phần tìm hiểu sau sẽ cần các thông tin tổng quá về toàn bộ dữ liệu. Để tiện lợi, nhóm sẽ tiền xử lý xóa dữ liệu trùng lắp và thiếu trước trên toàn bộ dữ liệu.

### Thử nghiệm xóa dữ liệu thiếu và trùng trước trên dữ liệu hiện tại

In [10]:
data_df.drop_duplicates(inplace=True)
data_df.dropna(axis=0, how="any", inplace=True)
data_df

,links,title,description,content,class
0,https://tuoitre.vn/tong-thong-trump-xac-nhan-k...,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,https://tuoitre.vn/dat-nuoc-dat-niem-tin-vao-n...,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,https://tuoitre.vn/luat-su-my-phan-bien-ong-tr...,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,https://tuoitre.vn/thanh-pho-phu-quoc-se-phat-...,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,https://tuoitre.vn/ong-trump-nguoi-ung-ho-toi-...,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới
...,...,...,...,...,...
51995,/tham-sat-o-thai-nguyen-nghi-pham-thua-nhan-nh...,Thảm sát ở Thái Nguyên: Nghi phạm thừa nhận nh...,"TTO - Tại cơ quan công an, bước đầu nghi phạm ...",Sau khi truy bắt Hoàng Văn Chín - nghi phạm gâ...,Pháp luật
51996,/mobifone-dat-loi-nhuan-hon-6-000-ty-dong-tron...,MobiFone đạt lợi nhuận hơn 6.000 tỉ đồng trong...,TTO – Mức lợi nhuận trước thuế của MobiFone ướ...,"Ngày 26-12, MobiFone công bố kết thúc năm 2019...",Kinh doanh
51997,/chuan-bi-co-le-hoi-hoa-anh-dao-tai-tp-hcm-201...,Chuẩn bị có lễ hội hoa anh đào tại TP.HCM,TTO - Làm việc với đoàn giám sát của HĐND TP.H...,"Ngày 26-12, Đoàn công tác của HĐND TP.HCM đã c...",Du Lịch
51998,/xet-xu-vu-nu-sinh-giao-ga-bui-van-cong-lien-t...,Xét xử vụ nữ sinh giao gà: Bùi Văn Công liên t...,"TTO - Được gọi lên xét hỏi, bị cáo Bùi Văn Côn...","Chiều 26-12, phiên tòa xét xử 9 bị cáo bắt cóc...",Pháp luật


### Chuẩn hóa unicode

In [ ]:
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    if (type(txt) is not str):
        return txt
    return unicodedata.normalize('NFC', txt)

### Chuẩn hóa dấu tiếng việt

In [ ]:
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    
    # Tách dấu ra khỏi từ và lưu lại vị trí nguyên âm
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1: # Phụ âm thì bỏ qua
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
                
        if y != 0: # Nếu có dấu
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
            
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            # Trường hợp chữ "gì"
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    # Từ hợp lệ là từ không có phụ âm bị kẹp giữa nguyên âm
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, _ = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if (index - nguyen_am_index) != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
    Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
    :param sentence:
    :return:
    """
    if (type(sentence) is not str):
        return sentence
    
    words = sentence.split()
    for index, word in enumerate(words):
        # Tách chữ có dính dấu ngăn cách câu để chuẩn hóa
        cw = re.sub(r'(^\p{P}*)([\p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        # Chuẩn hóa nếu chữ không có vấn đề (thường fail khi chưa tách thành công các từ)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
            words[index] = ''.join(cw)
    return ' '.join(words)

### Đoạn code thực hiện xóa và chuẩn hóa hàng loạt trên toàn dữ liệu

In [16]:
total_files = 218
text_attrs = ["title","description","content","class"]
prev_df = None

for index in tqdm(range(total_files)):
    df=pd.read_csv(f'src/scraped_data/crawling_{index}.csv')
    # Xóa dòng thiếu
    df.dropna(axis=0, how="any", inplace=True)
    # Xóa dòng trùng
    df.drop_duplicates(inplace=True)
    # Xóa dòng trùng với file trước
    if (prev_df is not None):
        df = df.merge(prev_df, how='outer', indicator=True).loc[lambda x : x['_merge'] == 'left_only']
        df.drop(['_merge'],axis=1,inplace=True)
    # Chuẩn hóa unicode và dấu
    for attr in text_attrs:
        df[attr]=df[attr].apply(covert_unicode)
        df[attr]=df[attr].str.lower()
        df[attr]=df[attr].apply(chuan_hoa_dau_cau_tieng_viet)
    df.to_csv(f'src/scraped_data/crawling_{index}.csv', index=False)
    prev_df = df

## 4. Tiếp tục khám phá

Trong phần này nhóm sẽ thực hiện thống kê trên toàn bộ dữ liệu, cụ thể là thống kê tổng số phân lớp của dữ liệu.

### Đoạn code thực hiện thống kê trên toàn dữ liệu

In [5]:
dir_4 = "src/scraped_data/"
tqdm.pandas()

In [6]:
# Import toàn bộ data (không khuyến cáo nếu máy không nhiều ram)
df = []
for i in range(218):
    df.append(pd.read_csv(dir_4 + f'crawling_{i}.csv'))
    
data_df=pd.concat(df)
data_df.reset_index(drop=True, inplace=True)

In [ ]:
# Thống kê tổng số phân lớp của toàn dữ liệu
data_df['class'].value_counts().to_csv("class_statistic.csv",encoding="utf-8", header=False)

### Thông kê phân lớp của toàn dữ liệu

In [17]:
class_stat = pd.read_csv("src/class_statistic.csv", header=None)
class_stat.columns = ["class", "count"]
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 2):
    display(class_stat)

,class,count
0,xã hội,107988
1,thế giới,78925
2,thể thao,69875
3,kinh doanh,53261
4,văn hóa,43199
5,nhịp sống trẻ,40813
6,pháp luật,38785
7,sức khỏe,30409
8,giáo dục,28048
9,cần biết,25922


Có thể thấy ở đây xuất hiện rất nhiều phân lớp (127 lớp), trong đó có khoảng 15 lớp có dưới 100 mẫu. Trong các lớp này xuất hiện một số lớp có chủ đề không rõ ràng (chẳng hạn như *"cần biết"* tại dòng 9) hay lớp có chủ đề không phù hợp với mục đích của nhóm (*"bạn đọc làm báo"* tại dòng số 11).

Nhóm quyết định sẽ loại bỏ toàn bộ phân lớp có dưới 500 mẫu. Sau đó sẽ tiến hành điều tra các phân lớp không phù hợp và đưa ra hướng giải quyết (loại bỏ hoặc chuyển phân lớp sang phân lớp khác phù hợp hơn).

Mục tiêu của nhóm sau khi điều tra là giảm được số phân lớp xuống trong khoảng 10-15 lớp.

In [18]:
# Phân lớp có số mẫu lớn hơn 500
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 2):
    display(class_stat[class_stat['count'] > 500])

,class,count
0,xã hội,107988
1,thế giới,78925
2,thể thao,69875
3,kinh doanh,53261
4,văn hóa,43199
5,nhịp sống trẻ,40813
6,pháp luật,38785
7,sức khỏe,30409
8,giáo dục,28048
9,cần biết,25922


### Điều tra phân lớp

Các phân lớp sau sẽ nằm trong diện điều tra của nhóm:

Phân lớp trùng lắp:
- thế giới, thời sự quốc tế
- xã hội, thời sự, phóng sự
- du lịch, đi chơi
- kinh doanh, tài chính, doanh nghiệp
- âm nhạc, giải trí, điện ảnh, tv show, văn hóa - giải trí
- nhịp sống số, khoa học, công nghệ, smarthome
- sức khỏe, phòng mạch, ăn gì, biết để khỏe
- giáo dục, tuyển sinh, học đường, câu chuyện giáo dục
    
Phân lớp không rõ ràng hoặc không phù hợp mục đích:
- nhịp sống trẻ
- cần biết
- bạn đọc làm báo
- tuôi trẻ cuôi tuần
- bình luận
- muôn màu
- phản hồi
- thủ thuật
- thư giãn
- kiều bào
- giả - thật
- đời sống
- mách bạn
- đường dây nóng
- mẹ & bé
- vấn đề - sự kiện
- tư vấn
- hồ sơ
- chuyên đề

### Xử lý các phân lớp trùng lắp

Sau khi kiểm tra, nhóm thấy các phân lớp đã liệt kê bên trên có nội dung tương tự nhau nên sẽ gom chung lại như sau:
- thế giới, thời sự quốc tế -> thời sự quốc tế.
- xã hội, thời sự, phóng sự -> thời sự trong nước.
- du lịch, đi chơi ->  du lịch.
- kinh doanh, tài chính, doanh nghiệp -> kinh doanh.
- âm nhạc, giải trí, tv show, điện ảnh -> giải trí.
- nhịp sống số, công nghệ -> công nghệ.
- nhà đất, smarthome -> nhà đất (ban đầu nhóm nghĩ smarthome thuộc về công nghệ nhưng khi điều tra thì không phải).
- sức khỏe, phòng mạch, biết để khỏe -> sức khỏe.
- giáo dục, tuyển sinh, học đường, câu chuyện giáo dục -> giáo dục.

Một số lớp sau không được gom nhóm ở bên trên:
- Lớp *văn hóa - giải trí* khó phân định giữa *văn hóa* hay *giải trí*, tuy nhiên do số lượng mẫu chỉ khoảng 1300, nhóm quyết định loại bỏ.
- Lớp *khoa học* chứa thông tin liên quan đến các ngành khoa học khác như sinh học, hóa học,... trong khi *nhịp sống số* và *công nghệ* chủ yếu chứ tin liên quan tới công nghệ thông tin. Nhóm quyết định tách lớp *khoa học* riêng với nhóm trên.
- Lớp *ăn gì* nhóm thấy phù hợp làm một nhóm mới hơn nên quyết định giữ lại làm lớp riêng.

In [45]:
data_df[data_df["class"] == "văn hóa - giải trí"]

,links,title,description,content,class
81645,https://tuoitre.vn/da-toi-luc-can-cac-chi-dan-...,đã tới lúc cần các chỉ dẫn đạo đức nghề nghiệp,ttct - sau những ồn ào tranh cãi liên quan tới...,chúng ta hãy bắt đầu từ việc quan sát nhiếp ản...,văn hóa - giải trí
81857,https://tuoitre.vn/ky-3-goc-khuat-trong-su-kie...,kỳ 3: góc khuất trong sự kiện sáng lập thương ...,ttct- trong quá trình tạo lập của thương hội h...,nguồn tư liệu về lịch sử thương hội không nhiề...,văn hóa - giải trí
82725,https://tuoitre.vn/ky-2-thuong-hoi-trung-hoa-d...,kỳ 2: thương hội trung hoa đầu tiên ở sài gòn ...,ttct - hội thương mại của người hoa ở chợ lớn ...,sách này chép “miếu quan đế do hội thương mại ...,văn hóa - giải trí
85130,https://tuoitre.vn/ai-noi-chuyen-kho-noi-14884...,ai nói chuyện khó nói?,ttct - khi đạo diễn rayka zehtabchi bước lên s...,nhưng cô muốn nói cho thật rõ với khán giả vì ...,văn hóa - giải trí
85546,https://tuoitre.vn/oscar-2019-nhung-nguoi-la-c...,oscar 2019: những người lạ chiếm chỗ,ttct - oscar - giải thưởng điện ảnh thường niê...,trong 24 hạng mục chính thức của giải oscar hằ...,văn hóa - giải trí
...,...,...,...,...,...
737879,https://tuoitre.vn/jimmy-wayne-am-nhac-tu-chin...,jimmy wayne: âm nhạc từ chính nỗi bất hạnh,"ttcn (tp.hcm) - 30 tuôi, được xem là tài năng ...",ca sĩ kiêm nhạc sĩ sáng tác jimmy wayne đã man...,văn hóa - giải trí
737882,https://tuoitre.vn/khai-hoan-mon-o-cac-nuoc-60...,khải hoàn môn ở các nước,ttcn - việc xây khải hoàn môn (khm) bắt nguồn ...,khm thứ nhất là titus được xây dựng để tôn vin...,văn hóa - giải trí
737891,https://tuoitre.vn/phat-hanh-phim-qua-internet...,phát hành phim qua internet,"ttcn - trong dịp cuôi tuần này, lần đầu tiên m...","phim nói về spike và heaton, kẻ cô đơn có tính...",văn hóa - giải trí
737895,https://tuoitre.vn/nhiem-sa-pa-594.htm,"nhiệm ""sa pa """,ttcn - nhiệm yêu những khoảnh khắc như chợt có...,có ai đó nói rằng để chụp ảnh một ngọn núi khô...,văn hóa - giải trí


In [46]:
data_df[data_df["class"] == "khoa học"]

,links,title,description,content,class
82,https://tuoitre.vn/dung-rau-buom-dem-tao-may-b...,máy bay biết... bay theo mùi,tto - đây được xem là bước ngoặt khi có thể kế...,khứu giác rất quan trọng với những loai bướm đ...,khoa học
86,https://tuoitre.vn/ret-dam-bao-trum-mien-bac-m...,"rét đậm bao trùm miền bắc, miền nam bao giờ tr...","tto - sáng 8-1, đợt không khí lạnh nhất từ đầu...","sáng sớm nay (8-1), không khí lạnh mạnh đã bao...",khoa học
99,https://tuoitre.vn/ha-giang-cung-xuat-hien-ban...,hà giang cũng xuất hiện băng giá,"tto - người dân cho biết sáng nay ở xín cái, h...","lúc 6h sáng nay 8-1, nhiệt độ ở xã xín cái, hu...",khoa học
115,https://tuoitre.vn/bang-gia-phu-trang-cao-bang...,"băng giá phủ trắng cao bằng, lạng sơn","tto - do ảnh hưởng của không khí lạnh, sáng sớ...",theo trung tâm dự báo khí tượng thủy văn quốc ...,khoa học
132,https://tuoitre.vn/ha-noi-hoa-toc-yeu-cau-ung-...,hà nội hỏa tốc yêu cầu ứng phó ô nhiễm không khí,"tto - tp hà nội yêu cầu các sở, ban ngành xây ...","ngày 7-1, ubnd tp hà nội có công văn hỏa tốc g...",khoa học
...,...,...,...,...,...
736469,https://tuoitre.vn/thuy-dien-tich-nang---giai-...,thủy điện tích năng - giải pháp mới cho nguồn ...,ttcn - nhu cầu sử dụng điện trong những năm gầ...,khác biệt so với thủy điện thông thườngvới thủ...,khoa học
737057,https://tuoitre.vn/chu-ky-nhiet-do-thay-doi-th...,chu kỳ nhiệt độ thay đổi theo tuần,tto - các thống kê thời tiết cho thấy mức chên...,hiện tượng thời tiết bất thường này không phải...,khoa học
737381,https://tuoitre.vn/hiem-hoa-tu-ca-noc-doc-1311...,hiểm họa từ cá nóc độc,"ttcn - do những quan niệm sai lầm, thiếu cơ sở...","đúng là thịt cá nóc có thể ăn được, nhưng phải...",khoa học
737387,https://tuoitre.vn/noi-oan-con-ca-loc-1306.htm,nỗi oan con cá lóc!,"ttcn - trong khi ở rất nhiều nơi cá lóc, cá rô...",hiện nay đi về các vùng ngọt hóa có phong trào...,khoa học


In [47]:
data_df[data_df["class"] == "ăn gì"]

,links,title,description,content,class
2955,https://tuoitre.vn/doc-dao-lua-chon-thit-bo-fu...,độc đáo lựa chọn thịt bò fujifoods làm quà giá...,"giáng sinh năm nay, bạn có thể gây ấn tượng vớ...","thịt bò cao cấp, quà tặng sang trọng giáng sin...",ăn gì
39071,https://tuoitre.vn/ga-nuong-o-o-o-ton-tai-vi-k...,"gà nướng ò ó o – tồn tại vì khác biệt, khác bi...",nếu như cách đây 5 năm gà nướng ò ó o chỉ là m...,gà nướng là món ăn quen thuộc và được ưa chuộn...,ăn gì
48846,https://tuoitre.vn/quan-co-gu-hard-rock-cafe-b...,quán có 'gu' hard rock cafe - bí kíp mở quán g...,kinh doanh nhà hàng khó ở chỗ phải tạo được né...,"lợi thế của mô hình có ""gu""nhà hàng có ""gu"" sẽ...",ăn gì
85423,https://tuoitre.vn/washoku-van-hoa-am-thuc-gia...,washoku - văn hóa ẩm thực giàu umami,"với văn hóa ẩm thực đặc trưng của nhật, washok...",văn hóa ẩm thực đặc trưng của người nhậtsáng t...,ăn gì
88299,https://tuoitre.vn/umami-tron-vi-ngon-ngay-tet...,umami - tròn vị ngon ngày tết,những món ăn đặc trưng của tết việt sẽ trọn vẹ...,vị umami là gì?là thành viên cở bản thứ năm tr...,ăn gì
...,...,...,...,...,...
691906,https://tuoitre.vn/4-mon-don-gian-cho-bua-tiec...,4 món đơn giản cho bữa tiệc đầu năm,"ngày tết làm món ăn nào vừa đơn giản, dễ làm m...",dành cho 4 người ănnguyên liệu: -cá điêu hồng ...,ăn gì
692035,https://tuoitre.vn/lam-mut-mang-cau-66094.htm,làm mứt mãng cầu,"mứt mãng cầu tương đối dễ làm, khi ăn có mùi c...",nguyên liệu - mãng cầu dai: 1 trái (1 kg)- đườ...,ăn gì
692037,https://tuoitre.vn/banh-chung-gac-don-tet-6563...,bánh chưng gấc đón tết,"cả xã duyên hà có đến 200-300 hộ, nghề làm bán...","bác khánh, mẹ anh thành cho biết, nhà lúc nào ...",ăn gì
692163,https://tuoitre.vn/nhung-cach-lau-rua-do-dung-...,những cách lau rửa đồ dùng thủy tinh,cọ rửa đồ dùng thủy tinh không đơn giản nếu nh...,"- dụng cụ thủy tinh đắt tiền, có thể dùng bàn ...",ăn gì


### Xử lý các phân lớp không rõ ràng hoặc không phù hợp

Hầu hết phân lớp trong danh sách trên đều trộn lẫn các tin của nhiều lớp khác, không rõ ràng. Tuy nhiên do số lượng mẫu không đáng kể nên sẽ loại bỏ đi. Ngoại trừ một số lớp sau:
- Lớp *nhịp sống trẻ* chứa thông tin về hoạt động sinh hoạt của sinh viên. Nhóm quyết định sẽ giữ lại.
- Lớp *hồ sơ* theo kiểm tra thì chứa hầu hết thông tin về chính trị quốc tế. Nhóm quyết định sẽ gom vào lớp *thời sự quốc tế*.

In [48]:
data_df[data_df["class"] == "nhịp sống trẻ"]

,links,title,description,content,class
102,https://tuoitre.vn/khu-vuon-mini-giua-long-sai...,khu vườn mini giữa lòng sài gòn,"tto - chỉ với 15m2, một khu vườn với nhiều loa...",mong muốn có không gian để gia đình cùng nhau ...,nhịp sống trẻ
131,https://tuoitre.vn/vinh-danh-sinh-vien-5-tot-v...,vinh danh sinh viên 5 tốt và sao tháng giêng,"tto - tối 7-1, trung ương hội sinh viên việt n...",chương trình kỷ niệm 71 năm ngày truyền thống ...,nhịp sống trẻ
163,https://tuoitre.vn/toi-yeu-to-quoc-toi-vao-top...,'tôi yêu tổ quốc tôi' vào top sự kiện tiêu biể...,tto - hội nghị ban thường vụ trung ương đoàn l...,"năm 2020, dịch covid-19 đã ảnh hưởng rất nhiều...",nhịp sống trẻ
304,https://tuoitre.vn/gioi-tre-voi-cau-chuyen-tha...,giới trẻ với câu chuyện thắt lưng buộc bụng - ...,tto - với nhiều bạn trẻ có hiểu biết về tín dụ...,với nhiều bạn trẻ có hiểu biết trong lĩnh vực ...,nhịp sống trẻ
445,https://tuoitre.vn/gioi-tre-voi-cau-chuyen-tha...,giới trẻ với câu chuyện thắt lưng buộc bụng - ...,"tto - thu nhập trên 20 triệu đồng mỗi tháng, n...","""nhà, xe tất nhiên ai cũng thích và tôi từng n...",nhịp sống trẻ
...,...,...,...,...,...
738150,https://tuoitre.vn/tin-dung-sinh-vien-de-nghi-...,tín dụng sinh viên: đề nghị tăng mức cho vay l...,(tt-tp.hcm) - trao đổi với tuôi trẻ qua điện t...,cụ thể: nâng mức vay từ 200.000 đồng/tháng lên...,nhịp sống trẻ
738161,https://tuoitre.vn/nhung-diem-tua-cho-tre-em-2...,những điểm tựa cho trẻ em,tt (tp.hcm) - các câu lạc bộ ông bà cháu ở khu...,với các nội dung giáo dục như kể chuyện truyền...,nhịp sống trẻ
738162,https://tuoitre.vn/bong-da-vi-tre-em-duong-pho...,bóng đá vì trẻ em đường phố,tt (tp.hcm) - trường vừa học vừa làm 15-5 cùng...,"mỗi đội ra sân với năm cầu thủ, ngoai ra còn c...",nhịp sống trẻ
738178,https://tuoitre.vn/tuong-trinh-tu-mot-manh-doi...,tường trình từ một mảnh đời,"tt (tp.hcm) - mảnh đời ấy tên lê bá kim, bị số...","một buôi trưa, chúng tôi đã tìm đến mái nhà “l...",nhịp sống trẻ


In [49]:
data_df[data_df["class"] == "hồ sơ"]

,links,title,description,content,class
23911,https://tuoitre.vn/25-nam-quan-he-ngoai-giao-v...,25 năm quan hệ ngoai giao việt - mỹ - kỳ 3: nh...,tto - thượng nghị sĩ mỹ william fulbright đã s...,"gs herbert covert, đại học colorado (mỹ), là m...",hồ sơ
32903,https://tuoitre.vn/eu-quan-ngai-truoc-cac-hanh...,eu quan ngại trước các hành động đơn phương ở ...,tto - đại sứ liên minh châu âu (eu) tại asean ...,đại sứ liên minh châu âu (eu) tại hiệp hội các...,hồ sơ
55417,https://tuoitre.vn/bat-them-3-nghi-pham-trong-...,bắt thêm 3 nghi phạm trong vụ 39 thi thể ở anh,tto - cảnh sát điều tra vụ việc tìm thấy 39 th...,"theo hãng tin reuters, trong khi các chuyên gi...",hồ sơ
65025,https://tuoitre.vn/hiep-uoc-luc-luong-hat-nhan...,hiệp ước lực lượng hạt nhân tầm trung - kỳ 4: ...,"tto - tháng 2-2019, mỹ thông báo ngừng thực hi...","đồng thời, mỹ ra tối hậu thư trong sáu tháng n...",hồ sơ
82614,https://tuoitre.vn/the-gioi-hoang-mang-vi-boei...,thế giới hoang mang vì boeing - kỳ 2: cuộc đua...,tto - nhiều người hẳn không quên vụ khủng bố k...,vụ tấn công khủng bố là một cú sốc đối với nướ...,hồ sơ
...,...,...,...,...,...
736757,https://tuoitre.vn/mien-vien-tay-cua-trung-don...,"""miền viễn tây"" của trung đông",tt - luật rừng đang ngự trị ở baghdad cũng như...,"người phụ nữ mặc áo trùm đen, ngồi thẳng người...",hồ sơ
736932,https://tuoitre.vn/nghi-can-sat-hai-ngoai-truo...,nghi can sát hại ngoai trưởng thụy điển là kẻ ...,tt - nghi can chính trong vụ án sát hại ngoai ...,tội trạng của nghi can này (hiện 35 tuôi) vẫn ...,hồ sơ
737317,https://tuoitre.vn/fso-va-cong-tac-bao-ve-v-pu...,fso và công tác bảo vệ v. putin,"tt - “kẻ ám sát luôn có cơ hội, và qui luật nà...","theo phân tích của fonarev, mấu chốt cuộc đối ...",hồ sơ
737419,https://tuoitre.vn/chinh-khach-tay-au-duoc-bao...,chính khách tây âu được bảo vệ ra sao?,tt - vụ ngoai trưởng thụy điển anna lindh bị đ...,pháp: bộ máy cận vệ khổng lồsau vụ tổng thống ...,hồ sơ


### Tổng kết các phân lớp của dữ liệu

1. thời sự quốc tế
2. thời sự trong nước
3. du lịch
4. kinh doanh
5. giải trí
6. công nghệ
7. nhà đất
8. sức khỏe
9. giáo dục
10. khoa học
11. thể thao
12. văn hóa
13. pháp luật
14. yêu
15. xe
16. thời trang
17. nhịp sống trẻ
18. ăn gì

## 5. Tiền xử lý sau cùng

### Xóa cột không cần thiết

Với mục đích của nhóm, các cột links, title và description đều không cần thiết.

In [7]:
data_df.drop(['links', 'title', 'description'],axis=1,inplace=True)

### Chỉnh sửa phân lớp và lọc phân lớp

Theo quyết định thay đổi phân lớp ở phần trên, nhóm sẽ viết hàm thực hiện chuyển đổi phân lớp.

In [8]:
def class_preprocess(df):
    selected_class = ["xã hội", "thế giới", "thể thao", "kinh doanh", "văn hóa", "pháp luật", "sức khỏe", "nhịp sống trẻ",
                      "giáo dục", "thời sự", "nhịp sống số", "tuyển sinh", "du lịch", "phóng sự", "nhà đất", "yêu",
                      "điện ảnh", "tài chính", "âm nhạc", "khoa học", "giải trí", "tv show", "công nghệ", "xe",
                      "thời trang", "smarthome", "đi chơi", "câu chuyện giáo dục", "hồ sơ", "thời sự quốc tế", "ăn gì"]
    processed_df = df[df["class"].isin(selected_class)].copy()
    processed_df.loc[processed_df["class"].isin(["thế giới", "hồ sơ"]), "class"] = "thời sự quốc tế"
    processed_df.loc[processed_df["class"].isin(["xã hội", "thời sự", "phóng sự"]), "class"] = "thời sự trong nước"
    processed_df.loc[processed_df["class"] == "đi chơi", "class"] = "du lịch"
    processed_df.loc[processed_df["class"].isin(["tài chính", "doanh nghiệp"]), "class"] = "kinh doanh"
    processed_df.loc[processed_df["class"] == "nhịp sống số", "class"] = "công nghệ"
    processed_df.loc[processed_df["class"].isin(["âm nhạc", "tv show", "điện ảnh"]), "class"] = "giải trí"
    processed_df.loc[processed_df["class"] == "smarthome", "class"] = "nhà đất"
    processed_df.loc[processed_df["class"].isin(["phòng mạch", "biết để khỏe"]), "class"] = "sức khỏe"
    processed_df.loc[processed_df["class"].isin(["tuyển sinh", "học đường", "câu chuyện giáo dục"]), "class"] = "giáo dục"
    return processed_df

In [9]:
class_preprocess(data_df)["class"].value_counts()

thời sự trong nước    142809
thời sự quốc tế        80010
thể thao               69875
kinh doanh             60805
văn hóa                43199
giáo dục               41822
nhịp sống trẻ          40813
pháp luật              38785
sức khỏe               30409
giải trí               24718
công nghệ              20825
du lịch                13208
nhà đất                10639
yêu                     8922
khoa học                6078
xe                      2823
ăn gì                   2286
thời trang              1605
Name: class, dtype: int64

In [10]:
data_df = class_preprocess(data_df)

In [3]:
!pip3 install pandarallel
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
!pip3 install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Traceback (most recent call last):
  File "/home/vlozg/.local/bin/jupyter", line 8, in <module>
    sys.exit(main())
  File "/home/vlozg/.local/lib/python3.8/site-packages/jupyter_core/command.py", line 285, in main
    command = _jupyter_abspath(subcommand)
  File "/home/vlozg/.local/lib/python3.8/site-packages/jupyter_core/command.py", line 124, in _jupyter_abspath
    raise Exception(
Exception: Jupyter command `jupyter-labextension` not found.


### Xóa ký tự đặc biệt

In [11]:
%%time
#xoá ký tự thừa
def remove_unnecessary(paragraph):
    temp=re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',paragraph)
    return re.sub(r'\s+', ' ', temp).strip()

data_df['content']=data_df['content'].parallel_apply(remove_unnecessary)

CPU times: user 13.4 s, sys: 5.64 s, total: 19 s
Wall time: 1min 22s


### Tokenize các từ trong câu và xóa stopwords

In [13]:
%%time
tqdm.pandas()
data_df['content']=data_df['content'].parallel_apply(ViTokenizer.tokenize)
#data_df['content']=data_df['content'].progress_apply(lambda x: word_tokenize(x, format="text"))

Process ForkPoolWorker-16:
Process ForkPoolWorker-19:
Process ForkPoolWorker-22:
Process ForkPoolWorker-23:
Process ForkPoolWorker-20:
Process ForkPoolWorker-24:
Process ForkPoolWorker-18:
Process ForkPoolWorker-21:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/li

In [14]:
with open('src/vietnamese-stopwords-dash.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    
# Danh sách stopword
stopword = set(lines)

In [15]:
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

In [ ]:
%%time
data_df['content']=data_df['content'].parallel_apply(remove_stopwords)

## 6. Tạo pipe tiền xử lý cho đoạn văn bản bất kỳ

## 7. Tách tập train, tập test và tập validation

In [ ]:
y_sr = data_df["class"] # sr là viết tắt của series
X_df = data_df["content"]
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3,stratify=y_sr, random_state=0)

# Feature Engineering đưa dữ liệu dạng văn bản đã được xử lý về dạng vector thuộc tính có dạng số học bằng TF-IDF Vectors ở mức Word level 

In [ ]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(train_X_df) # learn vocabulary and idf from training set
train_X_tfidf =  tfidf_vect.transform(train_X_df)
# assume that we don't have test set before
val_X_tfidf =  tfidf_vect.transform(val_X_df)

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(train_X_tfidf)


train_X_tfidf_svd = svd.transform(train_X_tfidf)
val_X_tfidf_svd = svd.transform(val_X_tfidf)

In [ ]:
train_X_tfidf_svd

# Xây dựng mô hình

In [ ]:
encoder =LabelEncoder()
train_y_sr_n = encoder.fit_transform(train_y_sr)
val_y_sr_n = encoder.fit_transform(val_y_sr)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg_model=LogisticRegression()
log_reg_model.fit(train_X_tfidf_svd,train_y_sr_n)


### Độ chính xác tập training

In [ ]:
log_reg_model.score(train_X_tfidf_svd,train_y_sr_n)

### Độ chính xác tập validation

In [ ]:
prediction=log_reg_model.predict(val_X_tfidf_svd)

In [ ]:
1 - (prediction != val_y_sr_n).mean()

# Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC
svm_classifier=SVC()
svm_classifier.fit(train_X_tfidf_svd,train_y_sr_n)

### Độ chính xác tập training

In [ ]:
svm_classifier.score(train_X_tfidf_svd,train_y_sr_n)

### Độ chính xác tập validation

In [ ]:
prediction=svm_classifier.predict(val_X_tfidf_svd)
1 - (prediction != val_y_sr_n).mean()

# MLPClassifier

In [ ]:
mlp_classifier=MLPClassifier(hidden_layer_sizes=(20), activation='tanh',
                                 solver='lbfgs', random_state=0, max_iter=2500)
mlp_classifier.fit(train_X_tfidf_svd,train_y_sr_n)

### Độ chính xác tập training

In [ ]:
mlp_classifier.score(train_X_tfidf_svd,train_y_sr_n)

### Độ chính xác tập validation

In [ ]:
prediction=mlp_classifier.predict(val_X_tfidf_svd)
1 - (prediction != val_y_sr_n).mean()